In [5]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=a1a2b5f197e472acd33a44099d083d9df48f5cb001e036777853a5af3a583442
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [13]:
import html
import re
import unicodedata
import numpy as np
from langdetect import detect, DetectorFactory
from tqdm import tqdm

# --- CẤU HÌNH ---
RAW_SRC = "/kaggle/input/iwslt15-englishvietnamese/IWSLT'15 en-vi/train.en.txt"
RAW_TRG = "/kaggle/input/iwslt15-englishvietnamese/IWSLT'15 en-vi/train.vi.txt"
FINAL_SRC = "train.final.en"
FINAL_TRG = "train.final.vi"

URL_REGEX = r'https?://\S+|www\.\S+'
CONTROL_CHAR_REGEX = r'[\x00-\x1f\x7f-\x9f\u200b\ufeff\xa0]' # Ký tự control, ZWS, BOM, NBSP
VN_CHAR_REGEX = re.compile(r'[áàảãạăằẳẵặâầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵ]')

MAX_LEN_EN = 80
MAX_LEN_VI = 90
MIN_LEN = 2

# --- 1. HÀM CLEAN TEXT NÂNG CAO (ĐÃ CẬP NHẬT) ---
def clean_text_final(text):
    if not isinstance(text, str): return ""
    
    # 1. Giải mã HTML entities
    text = html.unescape(text)
    
    # 2. Xóa HTML Tags
    text = re.sub(r'<[^>]+>', '', text) # Thêm logic này từ code bạn

    # 3. Xóa URL triệt để (Giả định URL_REGEX đã được định nghĩa)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # 4. Xóa Ký tự Control không in được (Giả định CONTROL_CHAR_REGEX đã được định nghĩa)
    # Tạm thay bằng ký tự control phổ biến
    text = text.replace('\xa0', ' ').replace('\u200b', '').replace('\ufeff', '')
    
    # 5. Chuẩn hóa Unicode về dạng NFC (CỰC KỲ QUAN TRỌNG)
    text = unicodedata.normalize('NFC', text)
    
    # 6. Chuẩn hóa/Xóa các loại dấu ngoặc đặc biệt thường là noise
    text = re.sub(r'[【】「」『』“”‘’“”]', '', text)

    # 7. Xóa metadata/tag trong ngoặc (Logic cũ)
    text = re.sub(r'\([^)]*\)', '', text) 
    text = re.sub(r'\[[^\]]*\]', '', text)
    
    # 8. Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 9. QUYẾT ĐỊNH CHO IWSLT (Để tăng tần suất học)
    text = text.lower() # <--- GIỮ LẠI BƯỚC NÀY CHO TẬP IWSLT NHỎ
    
    return text

In [16]:
def detect_safe(text):
    # Langdetect an toàn
    try:
        # Langdetect yêu cầu input là string, không phải list words
        return detect(text) if len(text) > 3 else 'unknown'
    except: 
        return 'unknown'
    
def is_vietnamese_text(text):
    # Kiểm tra Tiếng Việt trong cột En (dùng regex siêu nhanh)
    return bool(VN_CHAR_REGEX.search(text))

# --- HÀM CHECK TỈ LỆ ĐỘNG (LOGIC CỦA BẠN) ---
def is_valid_ratio_and_len(s_clean, t_clean):
    en_l = len(s_clean.split())
    vi_l = len(t_clean.split())
    
    if en_l == 0 or vi_l == 0: return False # Tránh chia 0
    ratio = vi_l / en_l # ratio = vi/en

    # 1. Lọc câu quá ngắn (min 3 từ)
    if en_l < MIN_LEN or vi_l < MIN_LEN:
        return False

    # 2. Lọc câu quá dài (Áp dụng ngưỡng 128/256)
    if en_l > MAX_LEN_EN or vi_l > MAX_LEN_VI:
        return False

    # 3. Lọc trường hợp tiếng Việt ngắn bất thường (Vi < 0.5 En)
    if ratio < 0.5:
        return False

    # 4. Ngưỡng theo độ dài câu tiếng Anh (Dynamic Ratio)
    if en_l < 5:          # câu rất ngắn
        return ratio <= 4.0
    if en_l < 12:         # câu ngắn - vừa
        return ratio <= 3.0

    # Câu dài (sử dụng ngưỡng 3.5 của bạn thay vì 2.5)
    return ratio <= 2.5

In [18]:
def process_final_data():
    print("🚀 BẮT ĐẦU XỬ LÝ DỮ LIỆU CHUẨN XÁC VỚI KIỂM TRA NGÔN NGỮ...")
    
    unique_pairs = set()
    stats = {"total": 0, "kept": 0, "duplicate": 0, "empty": 0, "bad_ratio": 0, "bad_lang": 0}
    
    with open(RAW_SRC, 'r', encoding='utf-8') as f_src, \
         open(RAW_TRG, 'r', encoding='utf-8') as f_trg, \
         open(FINAL_SRC, 'w', encoding='utf-8') as out_s, \
         open(FINAL_TRG, 'w', encoding='utf-8') as out_t:
        
        src_lines = f_src.readlines()
        trg_lines = f_trg.readlines()
        
        for s_line, t_line in tqdm(zip(src_lines, trg_lines), total=len(src_lines), desc="Processing"):
            stats["total"] += 1
            
            # 1. Clean Text
            s_clean = clean_text_final(s_line)
            t_clean = clean_text_final(t_line)
            
            # 2. Check Ratio & Length
            if not is_valid_ratio_and_len(s_clean, t_clean):
                stats["bad_ratio"] += 1
                continue
            
            # 3. Check Duplicate
            pair = f"{s_clean}\t{t_clean}"
            if pair in unique_pairs:
                stats["duplicate"] += 1
                continue
            unique_pairs.add(pair)
            
            # 4. KIỂM TRA NGÔN NGỮ
            
            # Tầng 1: Kiểm tra Tiếng Việt trong cột Anh (Fast check)
            if is_vietnamese_text(s_clean):
                 stats["bad_lang"] += 1
                 continue

            # Tầng 2: LangDetect (Clean check)
            lang_s = detect_safe(s_clean)
            lang_t = detect_safe(t_clean)
            
            is_valid_lang = (lang_s == 'en') and (lang_t == 'vi' or lang_t == 'unknown')
            
            if not is_valid_lang:
                stats["bad_lang"] += 1
                continue

            # --- GHI FILE ---
            out_s.write(s_clean + "\n")
            out_t.write(t_clean + "\n")
            stats["kept"] += 1

    print("\n✅ XỬ LÝ HOÀN TẤT!")
    print("-" * 40)
    print(f"Tổng ban đầu:   {stats['total']:,}")
    print(f"Giữ lại:        {stats['kept']:,} ({(stats['kept']/stats['total'])*100:.2f}%)")
    print(f"Bị loại:        {stats['total'] - stats['kept']:,}")
    print("-" * 40)
    print(f"Chi tiết loại bỏ:")
    print(f"  - Quá ngắn/Rỗng/Max Len/Ratio: {stats['bad_ratio']:,}")
    print(f"  - Trùng lặp:                   {stats['duplicate']:,}")
    print(f"  - Lỗi ngôn ngữ:                {stats['bad_lang']:,}")
    print("=" * 40)

# Chạy thôi
process_final_data()

🚀 BẮT ĐẦU XỬ LÝ DỮ LIỆU CHUẨN XÁC VỚI KIỂM TRA NGÔN NGỮ...


Processing: 100%|██████████| 133317/133317 [07:48<00:00, 284.30it/s]


✅ XỬ LÝ HOÀN TẤT!
----------------------------------------
Tổng ban đầu:   133,317
Giữ lại:        128,068 (96.06%)
Bị loại:        5,249
----------------------------------------
Chi tiết loại bỏ:
  - Quá ngắn/Rỗng/Max Len/Ratio: 737
  - Trùng lặp:                   912
  - Lỗi ngôn ngữ:                3,600


In [20]:
import random
import os

# --- CẤU HÌNH ĐƯỜNG DẪN ĐẾN FILE CUỐI CÙNG ---
FINAL_SRC = "train.final.en"
FINAL_TRG = "train.final.vi"

# --- 1. ĐỌC VÀ LẤY MẪU NGẪU NHIÊN ---
def get_random_samples(src_path, trg_path, num_samples=10):
    try:
        # Đọc toàn bộ nội dung file vào list
        with open(src_path, 'r', encoding='utf-8') as f:
            src_lines = f.readlines()
        with open(trg_path, 'r', encoding='utf-8') as f:
            trg_lines = f.readlines()
            
    except FileNotFoundError:
        print(f"❌ LỖI: Không tìm thấy file {src_path} hoặc {trg_path}. Vui lòng kiểm tra lại tên file.")
        return None

    total_count = len(src_lines)
    
    if total_count == 0:
        print("❌ LỖI: Tập dữ liệu rỗng sau khi xử lý!")
        return None

    # Lấy các chỉ số ngẫu nhiên
    if total_count < num_samples:
        indices = range(total_count)
    else:
        indices = random.sample(range(total_count), num_samples)

    print("=" * 60)
    print(f"✅ QA CHECK: Dữ liệu cuối cùng (Tổng {total_count:,} câu)")
    print(f"   In ra {len(indices)} mẫu ngẫu nhiên:")
    print("=" * 60)

    for i, idx in enumerate(indices):
        en_text = src_lines[idx].strip()
        vi_text = trg_lines[idx].strip()
        
        en_len = len(en_text.split())
        vi_len = len(vi_text.split())
        
        ratio = vi_len / en_len if en_len > 0 else 0
        
        print(f"[{i+1}/{len(indices)}] (Index gốc: {idx} | L: En={en_len}, Vi={vi_len} | Ratio={ratio:.2f})")
        print(f"  En: {en_text}")
        print(f"  Vi: {vi_text}")
        print("-" * 60)

    return total_count

# --- CHẠY KIỂM TRA ---
get_random_samples(FINAL_SRC, FINAL_TRG, num_samples=10)

✅ QA CHECK: Dữ liệu cuối cùng (Tổng 128,068 câu)
   In ra 10 mẫu ngẫu nhiên:
[1/10] (Index gốc: 125614 | L: En=39, Vi=45 | Ratio=1.15)
  En: and one of the things that we know is that , unfortunately , if you have a heart attack , then that tissue can start to die , and your outcome may not be very good over time .
  Vi: và một trong những điều mà chúng ta biết là , thật không may , nếu bạn lên cơn đau tim , sau đó , mô đó có thể bắt đầu chết , và hệ quả có lẽ sẽ không tốt theo thời gian .
------------------------------------------------------------
[2/10] (Index gốc: 111108 | L: En=22, Vi=19 | Ratio=0.86)
  En: it 's got germ . those are the good things , but those aren 't the tastiest parts of the wheat .
  Vi: trong bánh chứa vi khuẩn có lợi , nhưng đó không phải phần ngon nhất của hạt mỳ .
------------------------------------------------------------
[3/10] (Index gốc: 280 | L: En=15, Vi=22 | Ratio=1.47)
  En: if everyone in this room got involved , who knows what we could do ?
  Vi

128068